<a href="https://colab.research.google.com/github/ibacaraujo/cyclegan-seed/blob/master/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks

In [0]:
!pip install cadl

In [0]:
!pip install scipy==1.1.0

In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as tfl
from cadl.cycle_gan import lrelu, instance_norm

In [0]:
def encoder(x, n_filters=32, k_size=3, normalizer_fn=instance_norm, 
            activation_fn=lrelu, scope=None, reuse=None):
  with tf.variable_scope(scope or 'encoder', reuse=reuse):
    h = tf.pad(
        x, 
        [[0,0], [k_size, k_size], [k_size, k_size], [0, 0]], 
        "REFLECT")
    h = tfl.conv2d(
        inputs=h,
        num_outputs=n_filters,
        kernel_size=7,
        stride=1,
        padding='VALID',
        weights_initializer=tf.truncated_normal_initializer(stddev=0.02),
        biases_initializer=None,
        normalizer_fn=normalizer_fn,
        activation_fn=activation_fn,
        scope='1',
        reuse=reuse)
    h = tfl.conv2d(
        inputs=h,
        num_outputs=n_filters*2,
        kernel_size=k_size,
        stride=2,
        weights_initializer=tf.truncated_normal_initializer(stddev=0.002),
        biases_initializer=None,
        normalizer_fn=normalizer_fn,
        activation_fn=activation_fn,
        scope='2',
        reuse=reuse)
    h = tfl.conv2d(
        inputs=h,
        num_outputs=n_filters*4,
        kernel_size=k_size,
        stride=2,
        weights_initializer=tf.truncated_normal_initializer(stddev=0.02),
        biases_initializer=None,
        normalizer_fn=normalizer_fn,
        activation_fn=activation_fn,
        scope=3,
        reuse=reuse)
    return h

In [0]:
def residual_block(x, n_channels=128, normalizer_fn=instance_norm,
                   activation_fn=lrelu, kernel_size=3, scope=None, reuse=None):
  with tf.variable_scope(scope or 'residual', reuse=reuse):
    h = tf.pad(x, [[0,0], [1,1], [1,1], [0,0]], 'REFLECT')
    h = tfl.conv2d(
        inputs=h,
        num_outputs=n_channels,
        kernel_size=kernel_size,
        weights_initializer=tf.truncated_normal_initializer(stddev=0.02),
        biases_initializer=None,
        normalizer_fn=normalizer_fn,
        padding='VALID',
        activation_fn=activation_fn,
        scope='1',
        reuse=reuse)
    h = tf.pad(x, [[0,0], [1,1], [1,1], [0,0]], 'REFLECT')
    h = tfl.conv2d(inputs=h,
                   num_outputs=n_channels,
                   kernel_size=kernel_size,
                   weights_initializer=tf.truncated_normal_initializer(stddev=0.02),
                   biases_initializer=None,
                   normalizer_fn=normalizer_fn,
                   padding='VALID',
                   activation_fn=None,
                   scope='2',
                   reuse=reuse)
    h = tf.add(x, h)
  return h

In [0]:
def transform(x, img_size=256, reuse=None):
  h = x
  if img_size >= 256:
    n_blocks = 9
  else:
    n_blocks = 6
  for block_i in range(n_blocks):
    with tf.variable_scope('block_{}'.format(block_i), reuse=reuse):
      h = residual_block(h, reuse=reuse)
  return h